<a href="https://colab.research.google.com/github/KAYALAIMMANUELRAJU/PySpark1/blob/main/product_analytics_classicmodels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Case Study

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ClassicModelsProductAnalytics") \
    .getOrCreate()

print("✅ Spark Version:", spark.version)

✅ Spark Version: 3.5.1


In [2]:
from google.colab import files
uploaded = files.upload()

Saving classicmodels_data.zip to classicmodels_data.zip


In [ ]:
import zipfile
import os

with zipfile.ZipFile("classicmodels_data.zip", 'r') as zip_ref:
    zip_ref.extractall("classicmodels_data")  # Extract to a folder

# List extracted files
os.listdir("classicmodels_data")

In [7]:
base_path = "classicmodels_data/"

df_customers      = spark.read.option("header", "true").csv(base_path + "customers.csv", inferSchema=True)
df_employees      = spark.read.option("header", "true").csv(base_path + "employees.csv", inferSchema=True)
df_offices        = spark.read.option("header", "true").csv(base_path + "offices.csv", inferSchema=True)
df_orderdetails   = spark.read.option("header", "true").csv(base_path + "orderdetails.csv", inferSchema=True)
df_orders         = spark.read.option("header", "true").csv(base_path + "orders.csv", inferSchema=True)
df_payments       = spark.read.option("header", "true").csv(base_path + "payments.csv", inferSchema=True)
df_productlines   = spark.read.option("header", "true").csv(base_path + "productlines.csv", inferSchema=True)
df_products       = spark.read.option("header", "true").csv(base_path + "products.csv", inferSchema=True)

In [5]:
df_products.printSchema()
df_products.show(5)

root
 |-- productCode: string (nullable = true)
 |-- productName: string (nullable = true)
 |-- productLine: string (nullable = true)
 |-- productScale: string (nullable = true)
 |-- productVendor: string (nullable = true)
 |-- productDescription: string (nullable = true)
 |-- quantityInStock: string (nullable = true)
 |-- buyPrice: string (nullable = true)
 |-- MSRP: string (nullable = true)

+-----------+--------------------+------------+------------+--------------------+--------------------+---------------+--------+------+
|productCode|         productName| productLine|productScale|       productVendor|  productDescription|quantityInStock|buyPrice|  MSRP|
+-----------+--------------------+------------+------------+--------------------+--------------------+---------------+--------+------+
|   S10_1678|1969 Harley David...| Motorcycles|        1:10|     Min Lin Diecast|This replica feat...|           7933|   48.81|  95.7|
|   S10_1949|1952 Alpine Renau...|Classic Cars|        1:10|Cla

In [6]:
df_products.createOrReplaceTempView("products")
df_customers.createOrReplaceTempView("customers")
df_orders.createOrReplaceTempView("orders")

In [8]:
products = df_products.select("productCode", "productName", "productLine", "quantityInStock", "buyPrice", "MSRP")
customers = df_customers.select("customerNumber", "customerName", "contactLastName", "phone", "city", "creditLimit")
orders = df_orders.select("orderNumber", "customerNumber", "orderDate", "status", "comments", "shippedDate")

In [9]:
products.show(5)
customers.show(5)
orders.show(5)

+-----------+--------------------+------------+---------------+--------+------+
|productCode|         productName| productLine|quantityInStock|buyPrice|  MSRP|
+-----------+--------------------+------------+---------------+--------+------+
|   S10_1678|1969 Harley David...| Motorcycles|           7933|   48.81|  95.7|
|   S10_1949|1952 Alpine Renau...|Classic Cars|           7305|   98.58| 214.3|
|   S10_2016|1996 Moto Guzzi 1...| Motorcycles|           6625|   68.99|118.94|
|   S10_4698|2003 Harley-David...| Motorcycles|           5582|   91.02|193.66|
|   S10_4757| 1972 Alfa Romeo GTA|Classic Cars|           3252|   85.68| 136.0|
+-----------+--------------------+------------+---------------+--------+------+
only showing top 5 rows

+--------------+--------------------+---------------+------------+---------+-----------+
|customerNumber|        customerName|contactLastName|       phone|     city|creditLimit|
+--------------+--------------------+---------------+------------+---------+-

In [10]:
spark.sql("""
    CREATE OR REPLACE TEMP VIEW products_table AS
    SELECT productCode, productName, productLine, quantityInStock, buyPrice, MSRP
    FROM products
""")

DataFrame[]

In [11]:
spark.sql("""
    CREATE OR REPLACE TEMP VIEW customers_table AS
    SELECT customerNumber, customerName, contactLastName, phone, city, creditLimit
    FROM customers
""")

DataFrame[]

In [12]:
spark.sql("""
    CREATE OR REPLACE TEMP VIEW orders_table AS
    SELECT orderNumber, customerNumber, orderDate, status, comments, shippedDate
    FROM orders
""")

DataFrame[]

In [13]:
spark.sql("SELECT * FROM products_table").show(5)
spark.sql("SELECT * FROM customers_table").show(5)
spark.sql("SELECT * FROM orders_table").show(5)

+-----------+--------------------+------------+---------------+--------+------+
|productCode|         productName| productLine|quantityInStock|buyPrice|  MSRP|
+-----------+--------------------+------------+---------------+--------+------+
|   S10_1678|1969 Harley David...| Motorcycles|           7933|   48.81|  95.7|
|   S10_1949|1952 Alpine Renau...|Classic Cars|           7305|   98.58| 214.3|
|   S10_2016|1996 Moto Guzzi 1...| Motorcycles|           6625|   68.99|118.94|
|   S10_4698|2003 Harley-David...| Motorcycles|           5582|   91.02|193.66|
|   S10_4757| 1972 Alfa Romeo GTA|Classic Cars|           3252|   85.68| 136.0|
+-----------+--------------------+------------+---------------+--------+------+
only showing top 5 rows

+--------------+--------------------+---------------+------------+---------+-----------+
|customerNumber|        customerName|contactLastName|       phone|     city|creditLimit|
+--------------+--------------------+---------------+------------+---------+-

In [14]:
# Display 3 sample products
spark.sql("SELECT * FROM products_table LIMIT 3").show()

# Display 3 sample customers
spark.sql("SELECT * FROM customers_table LIMIT 3").show()

# Display 3 sample orders
spark.sql("SELECT * FROM orders_table LIMIT 3").show()

+-----------+--------------------+------------+---------------+--------+------+
|productCode|         productName| productLine|quantityInStock|buyPrice|  MSRP|
+-----------+--------------------+------------+---------------+--------+------+
|   S10_1678|1969 Harley David...| Motorcycles|           7933|   48.81|  95.7|
|   S10_1949|1952 Alpine Renau...|Classic Cars|           7305|   98.58| 214.3|
|   S10_2016|1996 Moto Guzzi 1...| Motorcycles|           6625|   68.99|118.94|
+-----------+--------------------+------------+---------------+--------+------+

+--------------+--------------------+---------------+------------+---------+-----------+
|customerNumber|        customerName|contactLastName|       phone|     city|creditLimit|
+--------------+--------------------+---------------+------------+---------+-----------+
|           103|   Atelier graphique|        Schmitt|  40.32.2555|   Nantes|    21000.0|
|           112|  Signal Gift Stores|           King|  7025551838|Las Vegas|    718

In [16]:
df_orderdetails.createOrReplaceTempView("orderdetails")

In [17]:
spark.sql("""
    SELECT
        c.customerNumber,
        c.customerName,
        ROUND(SUM(od.quantityOrdered * od.priceEach), 2) AS total_order_value
    FROM orders o
    JOIN orderdetails od ON o.orderNumber = od.orderNumber
    JOIN customers c ON o.customerNumber = c.customerNumber
    GROUP BY c.customerNumber, c.customerName
    ORDER BY total_order_value DESC
""").show(10)

+--------------+--------------------+-----------------+
|customerNumber|        customerName|total_order_value|
+--------------+--------------------+-----------------+
|           141|Euro+ Shopping Ch...|        820689.54|
|           124|Mini Gifts Distri...|        591827.34|
|           114|Australian Collec...|        180585.07|
|           151|  Muscle Machine Inc|        177913.95|
|           119|   La Rochelle Gifts|        158573.12|
|           148|Dragon Souveniers...|        156251.03|
|           323|Down Under Souven...|        154622.08|
|           131|   Land of Toys Inc.|        149085.15|
|           187|      AV Stores, Co.|        148410.09|
|           450|The Sharp Gifts W...|        143536.27|
+--------------+--------------------+-----------------+
only showing top 10 rows



In [18]:
spark.sql("""
    SELECT productCode, productName, quantityInStock
    FROM products
    ORDER BY quantityInStock ASC
    LIMIT 5
""").show()

+-----------+--------------------+--------------------+
|productCode|         productName|     quantityInStock|
+-----------+--------------------+--------------------+
|   S12_4473|   1957 Chevy Pickup|      Rubber wheels"|
|   S12_3148|  1969 Corvair Monza|          hood opens|
|   S24_1444|  1970 Dodge Coronet| hood opens and r...|
|  S700_2610|The USS Constitut...|   sea sprite on bow|
|   S24_1046|1970 Chevy Chevel...|                1005|
+-----------+--------------------+--------------------+



In [19]:
df_payments.createOrReplaceTempView("payments")

In [20]:
spark.sql("""
    SELECT
        customerNumber,
        ROUND(SUM(amount), 2) AS total_payment
    FROM payments
    GROUP BY customerNumber
    ORDER BY total_payment DESC
    LIMIT 5
""").show()

+--------------+-------------+
|customerNumber|total_payment|
+--------------+-------------+
|           141|    715738.98|
|           124|    584188.24|
|           114|    180585.07|
|           151|    177913.95|
|           148|    156251.03|
+--------------+-------------+



In [21]:
spark.sql("""
    SELECT
        date_format(orderDate, 'yyyy-MM') AS month,
        COUNT(orderNumber) AS total_orders
    FROM orders
    GROUP BY month
    ORDER BY month
""").show(12, truncate=False)

+-------+------------+
|month  |total_orders|
+-------+------------+
|2003-01|5           |
|2003-02|3           |
|2003-03|6           |
|2003-04|7           |
|2003-05|6           |
|2003-06|7           |
|2003-07|7           |
|2003-08|5           |
|2003-09|8           |
|2003-10|18          |
|2003-11|30          |
|2003-12|9           |
+-------+------------+
only showing top 12 rows



In [22]:
spark.sql("""
    CREATE OR REPLACE TEMP VIEW high_value_customers AS
    SELECT customerNumber, customerName, creditLimit
    FROM customers
    WHERE creditLimit > 100000
""")

DataFrame[]

In [23]:
spark.sql("SELECT * FROM high_value_customers").show()

+--------------+--------------------+-----------+
|customerNumber|        customerName|creditLimit|
+--------------+--------------------+-----------+
|           114|Australian Collec...|   117300.0|
|           119|   La Rochelle Gifts|   118200.0|
|           124|Mini Gifts Distri...|   210500.0|
|           131|   Land of Toys Inc.|   114900.0|
|           141|Euro+ Shopping Ch...|   227600.0|
|           146|Saveley & Henriot...|   123900.0|
|           148|Dragon Souveniers...|   103800.0|
|           151|  Muscle Machine Inc|   138500.0|
|           157|Diecast Classics ...|   100600.0|
|           187|      AV Stores, Co.|   136800.0|
|           227|Heintze Collectables|   120800.0|
|           239|Collectable Mini ...|   105000.0|
|           249|  Amica Models & Co.|   113000.0|
|           259|Toms Spezialitäte...|   120400.0|
|           276|Anna's Decoration...|   107800.0|
|           278|       Rovelli Gifts|   119600.0|
|           286|Marta's Replicas Co.|   123700.0|


In [24]:
order_value_df = spark.sql("""
    SELECT
        c.customerNumber,
        c.customerName,
        ROUND(SUM(od.quantityOrdered * od.priceEach), 2) AS total_order_value
    FROM orders o
    JOIN orderdetails od ON o.orderNumber = od.orderNumber
    JOIN customers c ON o.customerNumber = c.customerNumber
    GROUP BY c.customerNumber, c.customerName
    ORDER BY total_order_value DESC
""")

order_value_df.write.option("header", "true").mode("overwrite").csv("/content/output/order_value_by_customer")

In [25]:
top_customers_df = spark.sql("""
    SELECT
        customerNumber,
        ROUND(SUM(amount), 2) AS total_payment
    FROM payments
    GROUP BY customerNumber
    ORDER BY total_payment DESC
    LIMIT 5
""")

top_customers_df.write.option("header", "true").mode("overwrite").csv("/content/output/top_5_customers")

In [26]:
!zip -r classicmodels_outputs.zip /content/output/
from google.colab import files
files.download("classicmodels_outputs.zip")

  adding: content/output/ (stored 0%)
  adding: content/output/top_5_customers/ (stored 0%)
  adding: content/output/top_5_customers/._SUCCESS.crc (stored 0%)
  adding: content/output/top_5_customers/part-00000-c3d4b5f4-4955-455b-9e0e-d15913cbe100-c000.csv (deflated 11%)
  adding: content/output/top_5_customers/.part-00000-c3d4b5f4-4955-455b-9e0e-d15913cbe100-c000.csv.crc (stored 0%)
  adding: content/output/top_5_customers/_SUCCESS (stored 0%)
  adding: content/output/order_value_by_customer/ (stored 0%)
  adding: content/output/order_value_by_customer/._SUCCESS.crc (stored 0%)
  adding: content/output/order_value_by_customer/part-00000-eb1960ce-8fda-4226-81b3-07b9e951b479-c000.csv (deflated 47%)
  adding: content/output/order_value_by_customer/_SUCCESS (stored 0%)
  adding: content/output/order_value_by_customer/.part-00000-eb1960ce-8fda-4226-81b3-07b9e951b479-c000.csv.crc (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>